#Transfer learing with a pretrained ConvNet

在本教程中，您将学习如何通过使用来自预训练网络的转移学习来对猫和狗的图像进行分类。
预先训练的模型是一个保存的网络，以前是在大型数据集上进行训练的，通常是在大型图像分类任务上进行训练的。您可以按原样使用预训练的模型，也可以使用转移学习针对给定任务自定义该模型。
转移学习进行图像分类的直觉是，如果在足够大和足够通用的数据集上训练模型，则该模型将有效地充当视觉世界的通用模型。然后，您可以利用这些学习的功能图，而不必通过在大型数据集上训练大型模型而从头开始。

在本笔记本中，您将尝试两种方法来定制预训练的模型：

1. 特征提取：使用先前网络学到的表示法从新样本中提取有意义的特征。您只需在预先训练的模型之上添加一个新的分类器，即可从头开始对其进行训练，从而可以重新利用先前为数据集学习的特征图。

  您不需要（重新）训练整个模型。基本的卷积网络已经包含了通常用于图片分类的功能。但是，预训练模型的最终分类部分特定于原始分类任务，随后特定于训练模型的一组类。

2. 精调：取消冻结冻结模型基础的几个顶层，并共同训练新添加的分类器层和基础模型的最后一层。这使我们可以“微调”基本模型中的高阶特征表示，以使其与特定任务更加相关。

您将遵循一般的机器学习工作流程。

1. 检查并了解数据
2. 建立输入管道，在这种情况下，使用Keras ImageDataGenerator
3. 组成模型
  * 预训练基本模型中的载荷（和预训练权重）
  * 将分类层堆叠在顶部
4. 训练模型
5. 评估模型

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals

import os
import numpy as np
import matplotlib.pyplot as plt


In [0]:
import tensorflow as tf
import tensorflow.keras as keras

##数据预处理
###资料下载
使用TensorFlow数据集加载猫和狗数据集。

此tfds程序包是加载预定义数据的最简单方法。如果您拥有自己的数据，并且对与TensorFlow一起使用导入感兴趣，请参阅加载图像数据

In [0]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

该`tfds.load`方法下载并缓存数据，然后返回一个`tf.data.Dataset`对象。这些对象提供了强大，高效的方法来处理数据并将其通过管道传递到模型中。

由于`"cats_vs_dogs"`未定义标准拆分，因此请使用子拆分功能将其分为（训练，验证，测试），分别包含80％，10％和10％的数据。

In [5]:
#数据总量是total_num_examples=23262
#(train)
#(raw_train, raw_validation, raw_test)
dataset, metadata = tfds.load(
    'cats_vs_dogs',
    #split=(tfds.Split.TRAIN, tfds.Split.VALIDATION,tfds.Split.TEST),
    with_info=True,
    as_supervised=True
)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/2.0.1. Subsequent calls will reuse this data.


In [11]:
  dataset['train'].take(1)

<TakeDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

生成的tf.data.Dataset对象包含(image, label)成对的图像，其中图像具有可变的形状和3个通道，并且标签为标量。

显示训练集中的前两个图像和标签：